In [1]:
import pandas as pd

# Load the data
df = pd.read_csv('PedInf.txt', delimiter='\t')

# Create edge list
edges = []
for _, row in df.iterrows():
    if pd.notna(row['MaleParent']):
        edges.append((row['MaleParent'], row['LineName']))
    if pd.notna(row['FemaleParent']):
        edges.append((row['FemaleParent'], row['LineName']))

edges_df = pd.DataFrame(edges, columns=["Parent", "Child"])

# Save edges to CSV for loading into cuGraph
edges_df.to_csv("edges.csv", index=False)


In [4]:
import cudf
import cugraph

# Load the edges into cuDF
edges_df = cudf.read_csv("edges.csv")

# Create an undirected graph directly
G = cugraph.Graph(directed=False)
G.from_cudf_edgelist(edges_df, source='Parent', destination='Child')

# Perform Louvain community detection
louvain_parts, modularity = cugraph.louvain(G)
louvain_parts = louvain_parts.to_pandas()

# Perform Leiden community detection (if available in your version of cuGraph)
try:
    leiden_parts = cugraph.leiden(G)
    leiden_parts = leiden_parts.to_pandas()
except AttributeError:
    print("Leiden algorithm is not available in this version of cuGraph.")

# Display the results
print("Louvain Community Detection")
print(louvain_parts.head(10))

if 'leiden_parts' in locals():
    print("Leiden Community Detection")
    print(leiden_parts.head(10))


/home/useradd/miniconda3/envs/rapids-24.02/lib/python3.10/site-packages/cugraph/structure/symmetrize.py:93: FutureWarning: Multi is deprecated and the removal of multi edges will no longer be supported from 'symmetrize'. Multi edges will be removed upon creation of graph instance.
  warnings.warn(


Leiden algorithm is not available in this version of cuGraph.
Louvain Community Detection
   partition       vertex
0         15    CP92-1084
1          5    CP95-1028
2         18    CP17-4205
3          7  CPCL01-7231
4         20    CP10-1457
5         24  CPCL06-3302
6         13    CP12-1033
7         17    CP90-1557
8          7    CP09-1817
9         20    CP14-2394
Leiden Community Detection


AttributeError: 'tuple' object has no attribute 'head'

In [5]:
import networkx as nx
import matplotlib.pyplot as plt

# Convert the cuGraph graph to a NetworkX graph for visualization
G_nx = G.to_networkx()

# Add Louvain community information to the NetworkX graph
louvain_communities = louvain_parts.set_index('vertex')['partition'].to_dict()
nx.set_node_attributes(G_nx, louvain_communities, 'louvain')

# Plot Louvain communities
plt.figure(figsize=(12, 8))
pos = nx.spring_layout(G_nx)
colors = [louvain_communities[node] for node in G_nx.nodes()]
nx.draw(G_nx, pos, node_color=colors, with_labels=True, node_size=50, cmap=plt.cm.jet)
plt.title('Louvain Community Detection')
plt.show()

# If Leiden communities were computed
if 'leiden_parts' in locals():
    leiden_communities = leiden_parts.set_index('vertex')['partition'].to_dict()
    nx.set_node_attributes(G_nx, leiden_communities, 'leiden')

    # Plot Leiden communities
    plt.figure(figsize=(12, 8))
    colors = [leiden_communities[node] for node in G_nx.nodes()]
    nx.draw(G_nx, pos, node_color=colors, with_labels=True, node_size=50, cmap=plt.cm.jet)
    plt.title('Leiden Community Detection')
    plt.show()


AttributeError: to_networkx